In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

ModuleNotFoundError: No module named 'networkx'

In [ ]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

In [ ]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [ ]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

In [ ]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

In [ ]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

In [ ]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

In [ ]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

In [ ]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

In [ ]:
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

# General remarks:
# 1. For now we apply all levers at timestamp 0, so at the beginning of the 200years of the simulation.
# It seems the levers can be also implemented after 67 or 133 years.
# 2. For now we don't specify "EWS_DaysToThreat" as this is a secondary concern for determining the 
# preferred/best policy. It may be worth checking different values only after we choose the best policy.
# 3. For now we select policies only with ring 4 interest in mind. Since they should come up with a shared preferred
# policy with ring 5, maybe those should be combined with a lever that benefits ring 5.
policies = [
    Policy(
        "policy 1",
        # Big RfR in most upstream location (based on the map in ring 1 (non-organic farming)) -
        # our client would prefer this, as they don't care about non-organic farming and it should 
        # significantly decrease flooding of ring 4 (town Gorssel).
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1}
            # **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5} - 
            # this was original "policy 1" - since the course manual states: (for RfR 
            #"Once activated, the project remains active.", I think ""0_RfR 1": 1, "0_RfR 2": 1" 
            # are redundant - but we should evaluate this.
        )
    ),
    Policy(
        "policy 2",
        # Second best alternative of RfR from ring 4 perspective. Some RfR made upstream, 
        # some downstream from them, but none on their area. We choose this to have some comparison for policy 1.
        **dict(
            get_do_nothing_dict(),
            **{"3_RfR 0": 1}
        )
    ),
    Policy(
        "policy 3",
        # Only hightening dikes in ring 4 by the middle height of 0.5m (5 decimeters).
        **dict(
            get_do_nothing_dict(),
            **{"A.4_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 4",
        # Only hightening dikes in ring 4 by maximum height of 1m for comparison with policy 3.
        **dict(
            get_do_nothing_dict(),
            **{"A.4_DikeIncrease 0": 10}
        )
    ),
    Policy(
        "policy 5",
        # policy 1 + policy 3 - trade-off between costs of projects and max safety. Note that making more RfR 
        # (outside of ring 4 though, as they don't want to give up land) 
        # and hightening the dikes in ring 4 by more than 0.5m would increase ring 4 safety even further, but
        # is unrealistic with limiteed budget and stakeholders' interests.
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "A.4_DikeIncrease 0": 5}
        )
    ),
]

In [ ]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

In [ ]:
experiments, outcomes = results

In [ ]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)